<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Gradual introduction to OpenMDAO and FAST-OAD

This series of tutorials aims to guide the user in how to write multidisciplinary problems using the OpenMDAO and FAST-OAD framework. Complementary information can be found in the [OpenMDAO documentation](https://openmdao.org/docs/) and in the [FAST-OAD documentation](https://fast-oad.readthedocs.io/en/v1.3.1/).

In order to do so we will consider a very simple multidisciplinary analysis under the form of the sizing of an aircraft with very basic formulae. This problem will be written in different formats - from very basic Python to FAST-OAD syntax - and the differences and advantages will be explained along the way. This problem has been thought to showcase the capabilities of the FAST-OAD framework not its sizing methodology, meaning the formula used are not representaive of what is used in FAST-OAD(-GA). The formulae used were largely inspired from the ones used for the sizing of a CS-23 aircraft but they have been tweaked to reduce the number of variables in the problem.

## Description of the problem

In this problem, we will size an aircraft according to a reduced set of requirements: 
* The **MTOW** (Maximum Take-Off Weight) will be computed based on the fuel consumed on a given mission defined as a carrying given **payload mass** over a given **range**. 
* The structural weight (**OWE**) will be divided into two parts: the wing which weight will be computed based on the **MTOW**, its **area** and **AR** (aspect ratio) and the rest of the structure computed as a fraction of the **MTOW**. 
* The **lift-to-drag ratio** (L/D) will be computed based on the cruise equilibrium and a very simple quadratic drag model. The **constant drag** part will be further split into two parts : the **wing profile drag** which will depend on the **wing area** and the **profile drag of the rest of the fuselage** which will be kept constant. The **induced drag coefficient**, as for itself, will be computed based on the wing **AR**. 
* Finally, the **wing area** will be computed based on the **MTOW** assuming a **constant wing loading**.

At this point, a remark can be made : the MTOW is both an output of the process and an input of one of its model. This is called a sizing loop, and we will need to iterate to find an MTOW for which both the input and the output is equal. Once the loop is converged, then we will have a sized aircraft. In this first case all input data are fixed to the value chosen by the designer but loops still have to be resolved (e.g. the loops mentionned above), this is what is called a multi-disciplinary analysis or MDA.

A visual illustration of the global process is presented below.

<div class="row">
  <div class="column">
    <img src="./img/global_process.PNG" width="600">
  </div>
</div>

For the reader already familiar with the world of aircraft design, another remark can be made : based on the value of the AR, the aircaft L/D might increase (and so the fuel consumption will decrease) however, as the AR increases so will the wing bending moment and thus, the wing mass and consequently the OWE. Based on those observations, we can guess there will be an "ideal" AR for which the aircraft will be able to fly its design mission with the lowest MTOW. This is a multi-disciplinary optimization or MDO and the aspect ratio will be seen as a design variable! This is nothing but an added layer to the MDA where instead of having only fixed parameter, we allow one to vary (here, the aspect ratio) and we observe the change in the converged aircraft at the end. The optimizer will then try different value of the aspect ratio until the optimal one is found.

A visual illustration of the MDO process is presented below.

<div class="row">
  <div class="column">
    <img src="./img/mdo.PNG" width="600">
  </div>
</div>

## Description of the models

In this section we will describe the model and formulae we will use for this problem. As it might have been guessed from the previous graph, we will organise them by module to echo what is done in FAST-OAD and to keep things organized.

### Geometry

The geometry module is very simple as it only contains the computation of the wing area $S_{w}$ which is:

$$S_{w} = \frac{MTOW}{(MTOW/S_w)_{data}}$$

The ratio $(MTOW/S_w)_{data}$ is called the wing loading.

### Aerodynamic

The main output of the aerodynamic module will be the lift-to-drag ratio (L/D or l_d in the following code) in cruise conditions. In order to calculate it we will compute the constant drag and the lift induced drag based on the lift equilibrium in cruise conditions. Then we will divide the lift by the sum of the two drag sources obtained. A graph representing the module is shown below.

<div class="row">
  <div class="column">
    <img src="./img/aerodynamic.PNG" width="600">
  </div>
</div>

The constant(or profile or zero-lift) drag will be computed as:

$$C_{d0} = \frac{C_{d0_{other}}*S_{w,ref}}{S_w} + c $$

where $C_{d0_{other}}$ and $c$ are constant representing respectively the profile drag from the aircraft without the wing and the ratio between the wing profile drag and its wet area, they were obtained based on a more thourough computation with  a different wing area $S_{w,ref}$. Since we want to take the wing area as a reference area for the rest of our computation we divide the classic formula by $S_w$ hence the shape shown above.

The k coefficient (lift-induced drag coefficient) will be computed using a formula from [1]:

$$k = \frac{1}{\pi * AR * e} $$

with e, the Oswald efficiency factor computed with:

$$e = 1.78(1-0.045*AR^{0.68})-0.64$$

We can now obtain the total drag using a simple quadratic model:

$$C_d = C_{d0} + C_{di} = C_{d0} + k * C_l^2$$

with $C_l$ obtained using the lift equilibrium in cruise conditions, with g the free-fall acceleration constant, $\rho$ the air density at the cruise altitude and $V$ the cruise speed.

$$C_l = \frac{MTOW * g}{0.5*\rho*V^2*S_w}$$

### Mass

Similarly the mass model is described in the following picture.

<div class="row">
  <div class="column">
    <img src="./img/mass.PNG" width="600">
  </div>
</div>

The wing mass will be computed using a modified version of the formula presented by [1] and originating from [2]:

$$
m_W = 96.948*\left[\left(\frac{5.7*MTOW}{10^5}\right)^{0.65}*AR^{0.57}*\left(\frac{S_W}{100}\right)^{0.61}*2.5\right]^{0.993}
$$

where MTOW is in lbf and $S_W$ is in ft²

Then, for the OWE we will used a formula similar to what is presented in [1], which we will modify to account for the fact that the wing mass is computed on its own and add said wing:

$$
OWE = MTOW * ( 0.41 + 0.0066 * ln( MTOW ) ) + m_W
$$

### Performance

For the computation of the fuel consumed during the mission, we will use the Breguet range equation re-arranged to make the fuel appear:

$$
m_{fuel} = (OWE + Payload) * \left(e^{\dfrac{Range * TSFC * (1/(L/D) * g)}{V_{\infty}}} - 1\right)
$$

In this formula, the range and the payload define the aircraft design mission by describing how much weight it need to carry over which distance. The $TSFC$, or thrust specific fuel consumption, is a technological factor that characterizes the fuel consumed by unit force produced.

We can now finally computed our new MTOW knowing that:

$$ MTOW = OWE + Payload + m_{fuel} $$

## Content of the tutorial

The following notebook are dedicated to the implementation of this problem under various form. In every case we will split the models in differents files and folder to keep it organized.

* [The second notebook](02_pure_python.ipynb) is dedicated to the python implementation of the base formula, how to define the sizing process without using any optimization modules and how to solve it and simulate an MDO with very simple tools.

* [The third notebook](03_scipy.ipynb) will show you how to use scipy to resolve the MDA and how to implement an MDO, still using scipy.

* [The forth notebook](04_OpenMDAO.ipynb) will explain how this all translates in OpenMDAO and it will explains the benefits of using this framework.

* [The fifth notebook](04_OpenMDAO.ipynb)'s aims is to show how to use the FAST-OAD framework and what more it brings compared to OpenMDAO.

## Bibliography
[1] Gudmundsson, Snorri. General aviation aircraft design: Applied Methods and Procedures. Butterworth-Heinemann, 2013.

[2] Nicolai, Leland M., and Grant E. Carichner. Fundamentals of aircraft and airship design, Volume 1–Aircraft Design. American Institute of Aeronautics and Astronautics, 2010.